In [ ]:
%%capture
!pip install optuna
!pip install imblearn
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import seaborn as sns
from imblearn.over_sampling import SMOTE, SVMSMOTE
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

In [3]:
# import tensorflow as tf

# print('Connecting to GPU \n')
# # List GPUs and set memory growth before initializing TensorFlow or Keras models
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         print("Memory growth enabled for GPUs.")
#     except RuntimeError as e:
#         print(f"Error setting memory growth: {e}")
# else:
#     print("No GPUs found.")

# if gpus:
#     strategy = tf.distribute.MirroredStrategy()
#     print("Mirrored Strategy initialized with GPUs.")
# else:
#     strategy = tf.distribute.get_strategy()  # Default strategy if no GPUs available
#     print("Using Default Strategy.")

In [4]:
# import os
# import tensorflow as tf

# # Initialize the TPU
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All TPU devices:", tf.config.list_logical_devices('TPU'))


# # Configure TensorFlow to use the TPU
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
# tf.config.optimizer.set_jit(True)

In [5]:
# import tensorflow as tf

# # List all physical devices, specifically looking for GPUs
# gpus = tf.config.list_physical_devices('GPU')
# print(f"GPUs available: {gpus}")


In [6]:
# import os

# # Create the directory explicitly with the absolute path
# os.makedirs("~/.kaggle", exist_ok=True)

# !cp "/kaggle/input/kaggle-token/kaggle (1).json" ~/kaggle.json
# # Copy the kaggle.json file from your Google Drive to the required location
# !cp ~/kaggle.json ~/.kaggle/

# # Set the correct permissions for the Kaggle API token
# !chmod 600 ~/.kaggle/kaggle.json


In [7]:
# !pip install kaggle
# !kaggle competitions download -c playground-series-s4e10

In [8]:
# from zipfile import ZipFile

# with ZipFile('/content/playground-series-s4e10.zip', 'r') as zip_ref:
#     zip_ref.extractall('/content')

In [ ]:

print(f'reading the csv files into pandas dataframes.')

try:
    train_df = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
    test_df = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
    sub_df = pd.read_csv('/kaggle/input/playground-series-s4e10/sample_submission.csv')
except Exception as e:
    print(f'could not load the csv files into pandas dataframes')


In [10]:
# train_df.info()

In [11]:
# test_df.info()

In [12]:
# train_df.isnull().any(), test_df.isnull().any()

In [13]:
cat_cols = train_df.select_dtypes('object').columns
num_cols = []
for col in train_df.select_dtypes(exclude='object').columns:
    if col not in ('id', 'loan_status'):
        num_cols.append(col)
# num_cols, cat_cols

In [14]:
# train_df['loan_status'].value_counts(normalize=True)

In [15]:

def box_plots_num_cols(df, columns):
    df = df.copy()
    base_width = 10
    base_height = 5
    rows = len(columns)
    cols = 1
    fig_width = cols * base_width
    fig_height = rows * base_height
    fig, axes = plt.subplots(rows, cols, figsize=(fig_width, fig_height))
    axes = axes.flatten() if rows > 1 else [axes]
    for i, col in enumerate(columns):
        axes[i].boxplot(df[col])
        axes[i].set_title(col)
    for j in range(i+1, len(axes)):
        axes[j].axis('off')
    plt.tight_layout()

In [16]:
# box_plots_num_cols(train_df, num_cols)

In [17]:


def remove_outliers(df:pd.DataFrame, cols, beta):
    df = df.copy()
    for col in cols:
#         print(f'removing {col} outliers:\n')
        data = df[col]
        iqr = stats.iqr(data)
        q1 = np.percentile(data, 25)
        q3 = np.percentile(data, 75)
        outlier_low = q1 - beta * iqr
        outlier_high = q3 + beta * iqr
#         print(f'outlier_low:{outlier_low}, outlier_high: {outlier_high}')
        df = df[(data >= outlier_low) & (data <= outlier_high)]
    return df


In [ ]:
print(f'removing outliers before oversampling')
df_clean = remove_outliers(train_df, num_cols, 1.5)
# box_plots_num_cols(df_clean, num_cols)
print('removed outliers')
df_clean.shape, train_df.shape

In [19]:
def hist_plot_cat_cols(df:pd.DataFrame(), cat_cols=cat_cols):
    df = df.copy()
    fig, axes_cat = plt.subplots(1, len(cat_cols), figsize=(20, 10))
    for i, col in enumerate(cat_cols):
        axes_cat[i].hist(df[col])
        axes_cat[i].set_title(col)

In [20]:
# hist_plot_cat_cols(train_df)

In [21]:
# hist_plot_cat_cols(df_clean)

In [ ]:


print('encoding the categorical features')
df_clean = df_clean.drop(columns=['id'])

ord_enc = OrdinalEncoder()
one_hot_enc = OneHotEncoder(sparse_output=False)

ordinal_cols = ['loan_grade']
one_hot_cols = [col for col in cat_cols if col != 'loan_grade']
remaining_cols = [col for col in df_clean.columns if col not in cat_cols]

encoder = ColumnTransformer(
            transformers=[
                ('ordinal_encoder', ord_enc, ordinal_cols),
                ('one_hot_encoder', one_hot_enc, one_hot_cols),
                ('passthrough', 'passthrough', remaining_cols )
            ]
)

encoded_data = encoder.fit_transform(df_clean)
one_hot_enc.fit(df_clean[one_hot_cols])
one_hot_encoded_cols = one_hot_enc.get_feature_names_out(one_hot_cols)
all_cols = ordinal_cols + list(one_hot_encoded_cols) + remaining_cols
df_encoded = pd.DataFrame(encoded_data, columns=all_cols)
df_encoded.info()

In [ ]:

print(f'oversampling using SVMSMOTE')
df_enc_copy = df_encoded.copy()

smote = SMOTE(random_state=32,  k_neighbors=10)
svm_smote = SVMSMOTE(random_state=32, k_neighbors=10,m_neighbors=10)

X = df_enc_copy.drop(columns=['loan_status'])
y = df_enc_copy.loc[:, 'loan_status']

X_smote, y_smote = smote.fit_resample(X, y)
X_svm_smote, y_svm_smote = svm_smote.fit_resample(X, y)
X_smote.shape, y_smote.shape, X_svm_smote.shape, y_svm_smote.shape

In [24]:
smote_df = X_smote
smote_df['loan_status'] = y_smote

svm_smote_df = X_svm_smote
svm_smote_df['loan_status'] = y_svm_smote


In [25]:
# c = [col for col in smote_df.columns if col not in one_hot_encoded_cols]
# box_plots_num_cols(smote_df, c)

In [26]:
# c2 = [col for col in svm_smote_df.columns if col not in one_hot_encoded_cols]
# box_plots_num_cols(svm_smote_df, c2)

In [27]:
# smote_corr = smote_df.corr()
# svm_smote_corr = svm_smote_df.corr()

In [28]:

def heatmap_corr(corr):
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title('Correlation Heatmap')
    plt.show()

In [29]:
# heatmap_corr(smote_corr)

In [30]:
# heatmap_corr(svm_smote_corr)

In [31]:
# high_smote_corr = smote_corr[(abs(smote_corr) > 0.7) & (abs(smote_corr) != 1.0)]
# heatmap_corr(high_smote_corr)

In [32]:
# high_svm_smote_corr = svm_smote_corr[(abs(svm_smote_corr) > 0.7) & (abs(svm_smote_corr) != 1.0)]
# heatmap_corr(high_svm_smote_corr)

In [ ]:
print('dropping the column person home ownership rent')
df_final = svm_smote_df.drop(columns=[ 'person_home_ownership_RENT'])
df_final.info()

In [ ]:

print('splitting the dataset into train and test')

sss = StratifiedShuffleSplit(n_splits=1, random_state=32, test_size=0.1)
for tr, te in sss.split(X, y):
    df_train = df_final.iloc[tr]
    df_test  = df_final.iloc[te]
target_col = 'loan_status'
features_cols = [col for col in df_train.columns if col != target_col]
df_train.shape, df_test.shape

In [ ]:


def prepare_data_for_tpu(df:pd.DataFrame, features_cols:list[str], target_col:str, batch_size:int=32, poly_degree:int=2, poly:bool=False, valid_split:bool=False,valid_split_size:float=0.2):

    df = df.copy()
    features = df[features_cols]
    target = df.loc[:, target_col]

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=poly_degree, include_bias=False, interaction_only=True)) if poly else None
    ])
    pipe.steps = [step for step in pipe.steps if step is not None]

    features_final = pipe.fit_transform(features)
    features_final_df = pd.DataFrame(features_final, columns=[f'feature_{i}' for i in range(features_final.shape[1])])


    features_tensor = tf.convert_to_tensor(features_final_df.values.astype('float32'), dtype=tf.float32)
    target_tensor = tf.convert_to_tensor(target.values.astype('int64'), dtype=tf.int64)

    dataset = tf.data.Dataset.from_tensor_slices((features_tensor, target_tensor))

    dataset_size = len(features)
    val_size = int(valid_split_size * dataset_size)
    if valid_split and (valid_split_size > 0.0):
        valid_dataset = dataset.take(val_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
        train_dataset = dataset.skip(val_size).shuffle(buffer_size=dataset_size-val_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

        result = (train_dataset, valid_dataset)
    else:
        dataset = dataset.shuffle(buffer_size=dataset_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

        result = (dataset, )
    return result

print('preparing the datasets for tpu and splitting them into train and valid datasets: \n')
try:
    train_dataset, valid_dataset = prepare_data_for_tpu(df_train, batch_size=64, features_cols=features_cols, target_col=target_col, poly=True, poly_degree=3, valid_split=True, valid_split_size=0.2)
except Exception as e:
    print('Could not prepare the datasets.')

In [109]:
def build_model(input_shape, units, n_hidden_layers, batch_norm, drop_out, regularize, initializers, drop_out_rate):
    model = keras.Sequential()
    model.add(keras.Input(shape=(input_shape,)))

    for _ in range(n_hidden_layers):
        layer_params = {'units': units, 'activation': activations.relu}

        if regularize:
            layer_params['kernel_regularizer'] = regularizers.L1()
        if initializers:
            layer_params['kernel_initializer'] = keras.initializers.glorot_normal()

        model.add(layers.Dense(**layer_params))

        if drop_out:
            model.add(layers.Dropout(rate=drop_out_rate))
        if batch_norm:
            model.add(layers.BatchNormalization())

        units = max(2, round(units / 2))  # Gradual reduction in units

    # Output layer
    model.add(layers.Dense(1, activation=activations.sigmoid))
    return model

def compile_model(model, optimizer_name:str='Adam', learning_rate:float=0.001):
    if optimizer_name == 'Adam':
        optimizer = optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'SGD':
        optimizer = optimizers.SGD(learning_rate=learning_rate)
    elif optimizer_name == 'Nadam':
        optimizer = optimizers.Nadam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=keras.losses.binary_crossentropy, metrics=['accuracy'])
    return model

def train_model(model, train_dataset, valid_dataset, epochs, batch_size):
    history = model.fit(
        train_dataset,
        validation_data=valid_dataset,
        batch_size=batch_size,
        epochs=epochs,
        shuffle=True,
        verbose=0
    )
    return history.history['val_loss'][-1], history.history['val_accuracy'][-1]


In [ ]:
# best_params

In [169]:
import tensorflow as tf
import keras
from keras import activations, layers, initializers, regularizers, optimizers, losses
import optuna
from keras import backend as K

input_shape = train_dataset.element_spec[0].shape[1]  # Assuming first element is the feature tensor

# Objective function for Optuna
def objective(trial: optuna.Trial):
    # Define hyperparameters
    epochs = trial.suggest_int('epochs', 26, 45 )
    units = trial.suggest_int('units', 100, 250, step=10)
    n_hidden_layers = trial.suggest_int('n_hidden_layers', 2, 15)
    batch_norm = trial.suggest_categorical('batch_norm', [True, False])
    drop_out = trial.suggest_categorical('drop_out', [True, False])
    regularize = trial.suggest_categorical('regularize', [True, False])
    initializers = trial.suggest_categorical('initializers', [True, False])
    drop_out_rate = trial.suggest_float('drop_out_rate', 0, 0.7)
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-3, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD', 'Nadam'])

    # Create optimizer object based on the selected name
   

    batch_size = 64

    # Build, compile, and train the model
    model = build_model(input_shape, units, n_hidden_layers, batch_norm, drop_out, regularize, initializers, drop_out_rate)
    model = compile_model(model, optimizer_name)
    val_loss, val_accuracy = train_model(model, train_dataset, valid_dataset, epochs, batch_size)

    return val_loss, val_accuracy


In [84]:

class EarlyStoppingCallbackMinimize:
    def __init__(self, patience:int=50) -> None:
        self.patience = patience
        self.best_value = None
    
    def __call__(self, study:optuna.study.Study, trial:optuna.trial.FrozenTrial) -> None:
#         print('callback is called')
        current_value = trial.values[0]

        if self.best_value is None:
            self.best_value = current_value
        elif current_value < self.best_value:
            self.best_value = current_value
            self.patience = self.patience
        else:
            self.patience -= 1
            if self.patience == 0:
                print('stopping the study')
                study.stop()
        
        
        

In [ ]:

n_trials=500
patience = 100
early_stopping_cb = EarlyStoppingCallbackMinimize(patience=patience)
print(f'creating and optimizing the study with no of trials = {n_trials} and patience = {patience}')
try:
    
    loan_approval_ann_study = optuna.create_study(
    storage='sqlite:///loan_approval_ann_study.db',
    study_name='loan_approval_ann_study',
    directions=['minimize', 'maximize'],
    load_if_exists=True
    )
    print(f'created the srudy with no of trials = {n_trials}')
    loan_approval_ann_study.optimize(objective,  n_trials=n_trials, show_progress_bar=True,callbacks=[early_stopping_cb], n_jobs=-1)
    print('study completed')
except Exception as e:
    print(f'could not create the study', e)


In [ ]:
resumed_study = loan_approval_ann_study
best_value = 0
for trial in resumed_study.best_trials:
    curr = trial.values[0]
    if curr >= best_value:
        best_value = curr

for trial in resumed_study.best_trials:
    n = trial.values[0]
    if n >= best_value:
        print(trial.params)
        best_params = trial.params
        print(trial.values)

In [ ]:
best_params

In [ ]:
final_train_dataset, = prepare_data_for_tpu(df_train, batch_size=64, features_cols=features_cols, target_col=target_col, poly=True, poly_degree=3, valid_split=False)
final_input_shape = final_train_dataset.element_spec[0].shape[1]  # Assuming first element is the feature tensor


In [ ]:

final_model = build_model(
    input_shape=final_input_shape,
    units=best_params['units'],
    n_hidden_layers=best_params['n_hidden_layers'],
    batch_norm=best_params['batch_norm'],
    drop_out=best_params['drop_out'],
    regularize=best_params['regularize'],
    initializers=best_params['initializers'],
    drop_out_rate=best_params['drop_out_rate'],
)
final_model.summary()

In [ ]:
final_input_shape

In [132]:
final_model = compile_model(final_model, optimizer_name=best_params['optimizer'], learning_rate=best_params['learning_rate'])

In [ ]:
history = final_model.fit(final_train_dataset, epochs=best_params['epochs'])

In [ ]:
history.history['accuracy'][-1], history.history['loss'][-1]

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


print('encoding the final test data set')
ord_enc_test = OrdinalEncoder()
one_hot_enc_test = OneHotEncoder(sparse_output=False)

test_cat_cols = test_df.select_dtypes('object').columns
ordinal_cols = ['loan_grade']
one_hot_cols = [col for col in test_cat_cols if col != 'loan_grade']
remaining_cols = [col for col in test_df.columns if col not in list(test_cat_cols)]

encoder_test = ColumnTransformer(
            transformers=[
                ('ordinal_encoder', ord_enc_test, ordinal_cols),
                ('one_hot_encoder', one_hot_enc_test, one_hot_cols),
                ('passthrough', 'passthrough', remaining_cols )
            ]
)

encoded_data = encoder_test.fit_transform(test_df)
one_hot_enc_test.fit(test_df[one_hot_cols])
one_hot_encoded_cols = one_hot_enc_test.get_feature_names_out(one_hot_cols)
all_cols = ordinal_cols + list(one_hot_encoded_cols) + remaining_cols
test_df_encoded = pd.DataFrame(encoded_data, columns=all_cols)
test_df_encoded.info()

In [155]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

def prepare_test_data_for_tpu(df: pd.DataFrame, features_cols: list[str], batch_size: int = 32, poly_degree: int = 2, poly: bool = False):

    df = df.copy()
    features = df[features_cols]

    # Define pipeline for scaling and optional polynomial features
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=poly_degree, include_bias=False, interaction_only=True)) if poly else None
    ])
    pipe.steps = [step for step in pipe.steps if step is not None]

    # Transform features
    features_final = pipe.fit_transform(features)
    features_final_df = pd.DataFrame(features_final, columns=[f'feature_{i}' for i in range(features_final.shape[1])])

    # Create tf.data.Dataset without shuffling, batching only
    dataset = tf.data.Dataset.from_tensor_slices(features_final_df.values)
    dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    return dataset


In [ ]:
final_test_dataset = prepare_test_data_for_tpu(test_df_encoded, batch_size=64, features_cols=features_cols,  poly=True, poly_degree=3)
y_pred = final_model.predict(final_test_dataset, batch_size=64)

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

def prepare_test_data_for_tpu(df: pd.DataFrame, features_cols: list[str], batch_size: int = 32, poly_degree: int = 2, poly: bool = False):
    df = df.copy()
    features = df[features_cols]

    # Define pipeline for scaling and optional polynomial features
    steps = [('scaler', StandardScaler())]
    if poly:
        steps.append(('poly', PolynomialFeatures(degree=poly_degree, include_bias=False, interaction_only=True)))
    
    # Create the pipeline
    pipe = Pipeline(steps)

    # Transform features
    features_final = pipe.fit_transform(features)
    
    # Create DataFrame from transformed features
    feature_names = [f'feature_{i}' for i in range(features_final.shape[1])]
    features_final_df = pd.DataFrame(features_final, columns=feature_names)

    # If your model expects a specific input shape or structure, modify here
    if final_model.input_shape[1:] == features_final_df.shape[1:]:
        # Create tf.data.Dataset without shuffling
        dataset = tf.data.Dataset.from_tensor_slices(features_final_df.values)  # Use .values to get a numpy array
        dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    else:
        raise ValueError("Feature shape does not match model input shape.")

    return dataset

# Prepare the test dataset
final_test_dataset = prepare_test_data_for_tpu(test_df_encoded, batch_size=64, features_cols=features_cols, poly=True, poly_degree=3)

# Predict using the final model
y_pred = final_model.predict(final_test_dataset)


In [ ]:
y_pred.ravel().shape

In [ ]:
submission_df = pd.DataFrame({
    'id':test_df_encoded['id'],
    'loan_status':y_pred.ravel()
})
submission_df.head()

In [166]:
submission_df.to_csv('submission.csv', index=False)